<a href="https://colab.research.google.com/github/SergeyShchus/Data-Analysis/blob/master/Notebook/GIS%20analysts/Apple_Health_to_Google_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!unzip export.zip

Archive:  export.zip
  inflating: apple_health_export/export.xml  
  inflating: apple_health_export/export_cda.xml  


In [7]:
from xml.etree import ElementTree
import pandas as pd
import datetime

In [16]:
tree = ElementTree.parse('/content/apple_health_export/export.xml')
root = tree.getroot()
records = root.findall('Record')

In [17]:
records_dict = {
    'type':[],
    'unit':[],
    'creationDate':[],
    'startDate':[],
    'endDate':[],
    'value':[]
}

for record in records:
    for attribute in records_dict.keys():
        attribute_value = record.get(attribute)
        records_dict[attribute].append(attribute_value)

In [19]:
df = pd.DataFrame(records_dict)
df.head()

,type,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierHeight,cm,2020-11-27 23:50:47 +0200,2020-11-27 23:50:47 +0200,2020-11-27 23:50:47 +0200,187
1,HKQuantityTypeIdentifierBodyMass,kg,2020-11-27 23:50:47 +0200,2020-11-27 23:50:47 +0200,2020-11-27 23:50:47 +0200,81
2,HKQuantityTypeIdentifierStepCount,count,2020-11-23 21:39:41 +0200,2020-11-23 21:28:38 +0200,2020-11-23 21:28:41 +0200,8
3,HKQuantityTypeIdentifierStepCount,count,2020-11-23 22:30:40 +0200,2020-11-23 22:19:38 +0200,2020-11-23 22:19:40 +0200,16
4,HKQuantityTypeIdentifierStepCount,count,2020-11-23 22:45:09 +0200,2020-11-23 22:34:06 +0200,2020-11-23 22:34:08 +0200,8


In [20]:
types_dict = {
    'HKCategoryTypeIdentifierMindfulSession': 'Mindful Session',
    'HKQuantityTypeIdentifierDistanceCycling': 'Cycling Distance',
    'HKQuantityTypeIdentifierDistanceSwimming': 'Swimming Distance',
    'HKQuantityTypeIdentifierDistanceWalkingRunning': 'Walking + Running Distance',
    'HKQuantityTypeIdentifierFlightsClimbed': 'Flights Climbed',
    'HKQuantityTypeIdentifierHeartRate': 'Heart Rate',
    'HKQuantityTypeIdentifierRestingHeartRate': 'Resting Heart Rate',
    'HKQuantityTypeIdentifierStepCount': 'Steps',
    'HKQuantityTypeIdentifierActiveEnergyBurned': 'Active Calories',
    'HKQuantityTypeIdentifierBasalEnergyBurned': 'Resting Calories',
    'HKQuantityTypeIdentifierWalkingHeartRateAverage': 'Walking Heart Rate Average'
}

In [22]:
def td_to_m(td):
    seconds = td.seconds + td.days * 24 * 60 * 60
    return seconds // 60

In [23]:
df = pd.DataFrame(records_dict)
df.columns = ['type', 'unit', 'date', 'start', 'end', 'value']
df = df[df['type'].isin(types_dict.keys())]
df['value'] = df['value'].astype(float)
df['date'] = df['date'].astype('datetime64')
df['date'] = df['date'].dt.date
df['start'] = df['start'].astype('datetime64')
df['end'] = df['end'].astype('datetime64')
df['unit'] = df['unit'].astype(str)

In [24]:
df_1 = df[df['unit'] == 'count/min']
df_1 = df_1.groupby(by=['date', 'type', 'unit'], as_index=False).agg({'start':'min',
                                                                      'end':'max',
                                                                      'value':'mean'})

df_2 = df[df['type'] == 'HKCategoryTypeIdentifierMindfulSession']
df_2['value'] = df_2['end'] - df_2['start']
df_2['value'] = df_2['value'].map(td_to_m)
df_2 = df_2.groupby(by=['date', 'type', 'unit'], as_index=False).agg({'start':'min',
                                                                     'end':'max',
                                                                     'value':'sum'})
df_3 = df[(df['unit'] != 'count/min') & (df['type'] != 'HKCategoryTypeIdentifierMindfulSession')]
df_3 = df_3.groupby(by=['date', 'type', 'unit'], as_index=False).agg({'start':'min',
                                                                      'end':'max',
                                                                      'value':'sum'})
df = pd.concat([df_1, df_2, df_3])

In [25]:
df['date'] = df['date'].astype(str)
df['type'] = df['type'].apply(lambda x: types_dict[x])
dates = df['date'].unique()

In [26]:
result = {
    'date': [],
    'Steps': [],
    'Walking + Running Distance': [],
    'Swimming Distance': [],
    'Cycling Distance': [],
    'Resting Calories': [],
    'Active Calories': [],
    'Flights Climbed': [],
    'Heart Rate': [],
    'Resting Heart Rate': [],
    'Walking Heart Rate Average': [],
    'Mindful Session': []
}

In [27]:
for date in dates:
    part = df[df['date'] == date]
    result['date'].append(date)
    for key in result.keys():
        if key == 'date':
            continue
        else:
            field = 'value'
        try:
            result[key].append(part[part['type'] == key][field].values[0])
        except IndexError:
            result[key].append(None)

In [28]:
result_df = pd.DataFrame(result)
result_df = result_df.round(2)
result_df = result_df.sort_values(by='date')

In [29]:
result_df

,date,Steps,Walking + Running Distance,Swimming Distance,Cycling Distance,Resting Calories,Active Calories,Flights Climbed,Heart Rate,Resting Heart Rate,Walking Heart Rate Average,Mindful Session
0,2020-11-23,53.0,0.03,None,None,None,NaN,NaN,None,None,None,None
1,2020-11-24,10235.0,3.38,None,None,None,134.0,5.0,None,None,None,None
2,2020-11-25,22955.0,5.44,None,None,None,567.0,9.0,None,None,None,None
3,2020-11-26,7636.0,4.62,None,None,None,NaN,3.0,None,None,None,None
4,2020-11-27,36288.0,5.65,None,None,None,1168.0,9.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
104,2021-03-07,35214.0,9.83,None,None,None,724.0,47.0,None,None,None,None
105,2021-03-08,43726.0,16.71,None,None,None,768.0,36.0,None,None,None,None
106,2021-03-09,14888.0,8.13,None,None,None,126.0,27.0,None,None,None,None
107,2021-03-10,42802.0,11.39,None,None,None,1087.0,45.0,None,None,None,None


In [33]:
result_df.to_excel('Apple.xls')